In [1]:
#include <concepts>
#include <iostream>
#include <vector>

## 3.1 concept

In [2]:
template<typename T> 
concept is_integral = std::is_integral_v<T>;

In [3]:
std::cout << is_integral<int> << std::endl;

1


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe0b7c17500


In [4]:
std::cout << is_integral<double> << std::endl;

0


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe0b7c17500


In [5]:
template<typename T> 
concept is_floating_point = std::is_floating_point_v<T>;

In [6]:
template<typename T>
concept is_real = is_integral<T> || is_floating_point<T>;

In [7]:
is_real<decltype(0.0)>

(bool) true


In [8]:
is_real<decltype(1)> 

(bool) true


In [9]:
template<typename T>
concept is_not_real = ! is_real<T>;

In [10]:
is_not_real<decltype(1)>

(bool) false


In [11]:
is_not_real<decltype(std::true_type{})>

(bool) true


## 3.2 requires

In [12]:
template<typename M>
concept Machine = requires(M m){
    m.powerUp();
    m.powerDown();
};

In [13]:
struct motor{
    void powerUp(){};
    void powerDown(){};
};

In [14]:
Machine<motor>;

(bool) true


In [15]:
struct Toy{
    double powerUp(){return 1.0;};
    double powerDown(){return 0;};
};

In [16]:
Machine<Toy>;

(bool) true


In [17]:
template<typename T>
concept C = requires{
    typename T::type;
    typename std::vector<T>;
};

In [18]:
struct Foo{ using type = float; };

In [19]:
C<Foo>;

(bool) true


In [20]:
sizeof(Foo);

(unsigned long) 1


In [21]:
template<typename T>
concept Movable = requires(T a, T b){
    { a = std::move(b) } noexcept ;
};

In [22]:
Movable<std::vector<float>>

(bool) true


In [23]:
template<typename T> 
concept C1  = requires(T t){
    {t.f()} -> std::same_as<int>;
    {t.g()} -> std::convertible_to<double>;
};

In [24]:
struct Bar{
    int f(){return 1;};
    double g(){return 0.0;};
};

In [25]:
C1<Bar>

(bool) true


In [26]:
template<typename T>
concept C2 = requires(T t){
    t.f();
    requires std::same_as<decltype((t.f())),int>;
    t.g();
    requires std::convertible_to<decltype((t.g())), double>;
};

In [27]:
C2<Bar> 

(bool) true


In [28]:
template<typename T> 
constexpr bool has_member_swap = requires(T a, T b){
    a.swap(b);
};

In [29]:
has_member_swap<std::vector<float>>;

(const bool) true


In [30]:
has_member_swap<int>;

(const bool) false


We get true or compilation time error when using `requires`.

In [32]:
constexpr bool has_int_member_swap = requires(int a, int b){
    a.swap(b);
};

input_line_35:3:6: error: member reference base type 'int' is not a structure or union
    a.swap(b);
    ~^~~~~


ename: evalue

Below is a wrong case:

In [33]:
template<size_t N> 
constexpr bool IsEven = requires{
    N % 2 == 0;
};

In [34]:
IsEven<2>

(const bool) true


In [35]:
IsEven<1>

(const bool) true


Below is a right case:

In [36]:
template<size_t N> 
constexpr bool Even = requires{
    requires (N % 2 == 0);
};

In [37]:
Even<2>

(const bool) true


In [38]:
Even<1>

(const bool) false


Some mechanism like `SFINAE` is used here when requires leads to compilation time error.

In [39]:
template <typename T> 
void clever_swap(T& a, T& b){
    if constexpr (requires(T a, T b){a.swap(b);}){
        a.swap(b);
    }else{
        std::swap(a, b);
    }
};

In [40]:
std::vector v1{1}, v2{2};

In [41]:
clever_swap(v1, v2);

(void) @0x7fe08e3fe030


In [42]:
v1

(std::vector<int, allocator<int> > &) { 2 }


In [43]:
v2

(std::vector<int, allocator<int> > &) { 1 }


In [44]:
int i1 = 1, i2 = 2;

In [45]:
clever_swap(i1, i2);

(void) @0x7fe08e3fe030


In [46]:
i1

(int) 2


In [47]:
i2

(int) 1


In [48]:
template<typename T>
constexpr bool self_plus = requires (T v){
    requires (typename T::value_type x){++x;};
};

input_line_51:3:5: error: requires expression in requirement body; did you intend to place it in a nested requirement? (add another 'requires' before the expression)
    requires (typename T::value_type x){++x;};
    ^
    requires


ename: evalue

`(typename T::value_type x){++x;}` is not a valid bool expression.

In [49]:
template<typename T>
constexpr bool self_plus =  requires (T v){
    requires requires (typename T::value_type x){++x;};
};

GCC accept the code above and `self_plus<int>` below is false, we do not get a compilation time error.

In [50]:
self_plus<int>

(const bool) false


In [51]:
struct Plus{
    using value_type = int;
};

In [52]:
self_plus<Plus>

(const bool) true
